# **Invariant Information Clustering**

---




# Setting up the Enviorement

Clone the repository and set up the enviorement

In [0]:
!git clone -l -s https://github.com/nickpezzotti1/IIC-v0 code
%cd code

Cloning into 'code'...
remote: Enumerating objects: 463, done.
remote: Total 463 (delta 0), reused 0 (delta 0), pack-reused 463
Receiving objects: 100% (463/463), 55.67 MiB | 25.00 MiB/s, done.
Resolving deltas: 100% (244/244), done.
/content/code


Most required packages come preinstalled in Google Colab's Python 2 env. We only have to downgrade our pytorch and torchvision.

Full list of packages and versions can be found [here](https://github.com/nickpezzotti1/IIC-v0/blob/master/package_versions.txt).

In [5]:
!python -m pip install -q torch==0.4.1 torchvision==0.2.1 # We have to downgrade

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\LiveandInspire\AppData\Roaming\Python\Python37\site-packages\pip\_internal\cli\base_command.py", line 186, in _main
    status = self.run(options, args)
  File "C:\Users\LiveandInspire\AppData\Roaming\Python\Python37\site-packages\pip\_internal\commands\install.py", line 258, in run
    isolated_mode=options.isolated_mode,
  File "C:\Users\LiveandInspire\AppData\Roaming\Python\Python37\site-packages\pip\_internal\commands\install.py", line 604, in decide_user_install
    if site_packages_writable(root=root_path, isolated=isolated_mode):
  File "C:\Users\LiveandInspire\AppData\Roaming\Python\Python37\site-packages\pip\_internal\commands\install.py", line 549, in site_packages_writable
    test_writable_dir(d) for d in set(get_lib_location_guesses(**kwargs))
  File "C:\Users\LiveandInspire\AppData\Roaming\Python\Python37\site-packages\pip\_internal\commands\install.py", line 549, in <genexpr>
    test_writable_dir(d) f

In [2]:
import pickle
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch
import matplotlib.pyplot as plt
import numpy as np

import code.archs as archs

ModuleNotFoundError: No module named 'torch'

# Creating the Model

## A) Train a model 

In [0]:
!export CUDA_VISIBLE_DEVICES=0 && python -m code.scripts.cluster.cluster_greyscale_twohead --out_root /content/out/adv --model_ind 687 --arch ClusterNet6cTwoHead --mode IID --dataset_root utils/cluster/MNIST.py --dataset MNIST-uniform-noise --gt_k 10 --output_k_A 10 --output_k_B 10 --lamb_A 1.0 --lamb_B 1.0 --lr 0.0001 --num_epochs 20 --batch_sz 700 --num_dataloaders 5 --num_sub_heads 5 --crop_orig --crop_other --tf1_crop centre_half --tf2_crop random --tf1_crop_sz 20 --tf2_crop_szs 16 20 24 --input_sz 24 --rot_val 25 --no_flip --head_B_epochs 2 --nu 2 --adv_path /content/pgd-adversarials.txt --adv_n 1300

## B) Load a pre-trained model

Upload your pickled model and config files. Change the path variables to import your own model. 

The following model and configs correspond to the pretrained MNIST model 685 trained by Xu Ji et al. at Oxford (2018).

In [0]:
config_path = "/content/code/mnist_original/config.pickle"
net_path = "/content/code/mnist_original/best_net.pytorch"

Load the model:

In [0]:
%%capture

config_in = open(config_path, "rb")
config = pickle.load(config_in)
net = archs.__dict__[config.arch](config)
net.load_state_dict(torch.load(net_path))

The following code allows us to inspect some of the key features of our pretrained model:

In [0]:
config_dict = config.__dict__
number_of_epochs = len(config_dict['epoch_acc'])
mappings = {pair[0]: pair[1] for pair in config.__dict__['epoch_stats'][number_of_epochs - 1]["best_train_sub_head_match"]}
inv_mappings = {pair[1]: pair[0] for pair in config.__dict__['epoch_stats'][number_of_epochs - 1]["best_train_sub_head_match"]}
best_head = config.__dict__['epoch_stats'][number_of_epochs - 1]["best_train_sub_head"]

print("Epochs", number_of_epochs)
print("Average", config.__dict__['epoch_stats'][number_of_epochs - 1]["avg"])
print("Best mapping", config.__dict__['epoch_stats'][number_of_epochs - 1]["best_train_sub_head_match"])
print("Best head", config.__dict__['epoch_stats'][number_of_epochs - 1]["best_train_sub_head"])

# Using the model

Load the MNIST dataset

In [0]:
dl = DataLoader(torchvision.datasets.MNIST(
    '/data/mnist',
    train=True, 
    download=True, 
    transform=transforms.Compose([
      transforms.ToTensor()
      ])), 
    batch_size=100)

Visualise the predictions of the model

In [0]:
dataloader_iter = enumerate(dl)

def display(iter, n):
    f, axarr = plt.subplots(1, n, figsize=(20,20))

    count = 0
    for batch_index, batch in dataloader_iter:
      for i in range(len(batch)):
        if count >= n:
          return
        
        image = batch[0][i]
        prediction = mappings[torch.argmax(
            net(image.reshape(1, 1, 28, 28))[best_head]).item()]
        label = batch[1][i].item()

        axarr[count].title.set_text("Pred " + str(prediction) + " for " + 
                                    str(label))

        axarr[count].imshow(image.reshape(28, 28))
        count += 1

    plt.show()

display(iter, 10)

In order to calculate the accuracy of our model we must run over a mini-batch. Because of batch normalization (see. https://github.com/xu-ji/IIC/issues/49 )
We have to test a batch at a time instead of testing one image at a time to observe max accuracy

In [0]:
dataloader_iter = enumerate(dl)

total = 0
correct = 0

for batch_index, batch in dataloader_iter:
  if (batch_index == 1): break
  softmax_predictions = net(batch[0])[best_head]
  predictions = np.array([torch.argmax(softmax_prediction) for 
                          softmax_prediction in softmax_predictions])
  labels = np.array([inv_mappings[label.item()] for label in batch[1]])

  total += len(labels)
  correct += np.sum(predictions == labels)


print("Accuracy: " + str(float(correct)/total))

In [0]:
f = open("/content/pgd-adversarials.txt", "rb")
      
dataset = pickle.load(f)
len(dataset)